# **Generate Lighting**

Per capire "a che serve", dobbiamo guardare a un problema fondamentale della computer grafica: **calcolare la luce è costoso**.

Se il tuo computer dovesse calcolare in tempo reale (60 volte al secondo) ogni singolo raggio di luce che rimbalza da una parete all'altra (Global Illumination), il gioco girerebbe a 2 frame al secondo, a meno che tu non abbia una scheda video di ultimissima generazione con Ray Tracing attivo.

Ecco cosa fa esattamente **Generate Lighting**:

### 1. Il Concetto: Spostare il calcolo dal "Durante" al "Prima"

Quando premi quel pulsante, Unity ferma il tempo e inizia a fare calcoli complessi. Lancia milioni di raggi virtuali nella tua scena per capire:

* Dove cade la luce diretta?
* Dove rimbalza la luce (luce indiretta)?
* Dove sono le zone d'ombra morbida (Ambient Occlusion)?

Una volta finito il calcolo, Unity prende il risultato e lo "dipinge" su delle texture speciali chiamate **Lightmaps**.

> **L'Analogia:**
> * **Luci Realtime:** È come camminare in una stanza buia con una torcia. La luce si muove con te, crea ombre dinamiche, ma richiede energia costante.
> * **Generate Lighting (Baking):** È come se tu prendessi della vernice nera e dipingessi le ombre direttamente sul pavimento e sui muri. Quando entri nella stanza, non c'è bisogno di calcolare l'ombra: è già disegnata lì. È "finta", ma sembra vera e non costa nulla in termini di calcolo.

### 2. Cosa succede tecnicamente? (Il dietro le quinte)

Quando clicchi "Generate Lighting", Unity esegue tre operazioni principali:

#### A. Creazione delle Lightmaps

Unity srotola la geometria dei tuoi oggetti 3D (usando le coordinate UV) e crea un file immagine (Texture) che contiene le informazioni di luce e ombra.

* **Risultato:** Durante il gioco, Unity non calcola luci e ombre per quegli oggetti. Semplicemente applica questa texture "sopra" quella del colore (Albedo).
* **Vantaggio:** Performance incredibili. Puoi avere luci bellissime e realistiche su un telefono di fascia bassa.

#### B. Calcolo della Global Illumination (GI)

La luce non si ferma quando colpisce un muro; rimbalza e illumina leggermente la parete opposta (Color Bleeding).

* **Esempio:** Se hai un pavimento rosso illuminato dal sole, le pareti bianche vicine si tingeranno leggermente di rosa.
* Il calcolo Realtime di questo effetto è quasi impossibile senza hardware costoso. Il **Baking** lo calcola perfettamente.

#### C. Generazione delle Light Probes

Se "cuociamo" le luci sui muri (che sono fermi), cosa succede al tuo personaggio che si muove? Se entra in una zona d'ombra "dipinta", non dovrebbe scurirsi?
Qui entrano in gioco le **Light Probes**: punti invisibili nello spazio che memorizzano "qui c'è luce" o "qui c'è ombra". Il personaggio, passando vicino a questi punti, capisce come illuminarsi. "Generate Lighting" calcola anche questi dati.

### Quando usare "Generate Lighting"?

Non sempre. Dipende dal tipo di luce che hai impostato nell'Inspector (`Mode` della luce):

1. **Realtime:** La luce viene calcolata ogni frame. "Generate Lighting" ignora queste luci per le ombre dirette, ma le usa per calcolare i rimbalzi indiretti se la GI è attiva.
2. **Baked:** La luce è totalmente statica. Non puoi muoverla, non puoi cambiarle colore durante il gioco. È quella che beneficia al 100% del "Generate Lighting".
3. **Mixed:** Il compromesso. La luce proietta ombre statiche sugli oggetti fermi (baking) e ombre dinamiche sugli oggetti che si muovono.

### Il prerequisito fondamentale: La spunta "Static"

Questo è l'errore numero uno di chi inizia.
Affinché "Generate Lighting" funzioni su un oggetto (es. un muro o un pavimento), devi dire a Unity che quell'oggetto **non si muoverà mai**.

Devi selezionare il GameObject e nell'Inspector, in alto a destra, spuntare la casella **Static**.
Se non lo fai, Unity ignorerà quell'oggetto durante il calcolo delle Lightmap, perché non può "dipingere" un'ombra su qualcosa che potrebbe spostarsi.


### Analisi delle implicazioni (Pro e Contro)

| Aspetto | Realtime Lighting (Senza Baking) | Baked Lighting (Generate Lighting) |
| --- | --- | --- |
| **Performance (CPU/GPU)** | Pesante (basso FPS su scene complesse) | Leggerissimo (alto FPS) |
| **Memoria (RAM/VRAM)** | Bassa | Alta (le Lightmap sono texture extra da caricare) |
| **Qualità Visiva** | Ombre nette, niente luce riflessa (di base) | Ombre morbide, luce riflessa realistica |
| **Dinamicità** | Puoi muovere le luci o spegnerle | Le luci sono "cementate" nella scena |

---

